<h1> IMDb Actor Actress Network </h1>

<h2> Util methods </h2>

In [2]:
import re
import numpy as np
import pickle
import itertools

In [2]:
def remove_brackets(var):
    #return re.sub("[\(].*?[\)]", "", var)
    #including year in movie
    i = var.find(")")
    
    if i == -1:
        return re.sub("[\(].*?[\)]", "", var)
    
    else:
        return var[:i+1] + re.sub("[\(].*?[\)]", "", var[i+1:])

def clean_actor_name(name):
    name = name.replace("&#378;","z")
    return name
    

#def total_length(list_of_lists):
#    return len(set(itertools.chain.from_iterable(([i[1:] for i in list_of_lists]))))

def total_length(dictionary):
    unique = set([])
    for k,v in dictionary.items():
        for i in v:
            unique.add(i)
    return len(unique)

In [3]:
def preprocess(actors):
    
    actor_movie_map = {}

    for act in actors:
        
        movies = act.split('\t\t')
        movie_names = map(remove_brackets, movies[1:])
        mov = list(set([movie.strip('\n').strip() for movie in movie_names]))
        
        if len(mov) >= 10:    
            actor_name =  clean_actor_name(movies[0])
            actor_movie_map[actor_name] = mov
            
    return actor_movie_map

<h2> Preprocessing and Insights </h2>

In [4]:
actors = list(open("project4_data/actor_movies.txt", "r"))
actress = list(open("project4_data/actress_movies.txt", "r"))

In [5]:
actors_movie_map = preprocess(actors)
actress_movie_map = preprocess(actress)
print("Considering people acted in >= 10 unique movies")
print("Total Number of Actors - ", len(actors_movie_map.keys()))
print("Total Number of Actress - ", len(actress_movie_map.keys()))

Considering people acted in >= 10 unique movies
Total Number of Actors -  74586
Total Number of Actress -  38528


In [6]:
actor_movie_map = {**actors_movie_map, **actress_movie_map}
print("Total number of actors and actresses - ", len(actor_movie_map.keys()))

Total number of actors and actresses -  113114


In [7]:
print("Unique Number of Actors Movies - ", total_length(actors_movie_map))
print("Unique Number of Actress Movies - ", total_length(actress_movie_map))
print("Unique Number of Overall Movies - ", total_length(actor_movie_map))

Unique Number of Actors Movies -  428998
Unique Number of Actress Movies -  328739
Unique Number of Overall Movies -  468458


In [8]:
num_movies = {}
for key, val in actor_movie_map.items():
    num_movies[key] = len(val)

In [9]:
movie_actor_map = {}

for actor, movies in actor_movie_map.items():
    
    for movie in movies:
        
        if movie_actor_map.get(movie):
            movie_actor_map[movie].append(actor)
        else:
            movie_actor_map[movie] = [actor]

In [10]:
actor_pairings = {}

for movie, actors in movie_actor_map.items():
    comb = map(tuple,map(sorted, list(itertools.combinations(actors, 2))))
    for c in comb:
        if actor_pairings.get(c):
            actor_pairings[c] += 1
        else:
            actor_pairings[c] = 1

In [11]:
movie_pairings = {}

for actor, movies in actor_movie_map.items():
    comb = map(tuple,map(sorted, list(itertools.combinations(movies, 2))))
    for c in comb:
        if movie_pairings.get(c):
            movie_pairings[c] += 1
        else:
            movie_pairings[c] = 1

In [11]:
edge_list = open('edge_weights_list.txt','a')

for pair, count in actor_pairings.items():

    edge_list.write('\t'.join([pair[0], pair[1], str(count / num_movies[pair[0]])]))    
    edge_list.write('\n') 
    edge_list.write('\t'.join([pair[1], pair[0], str(count / num_movies[pair[1]])]))     
    edge_list.write('\n')

In [14]:
def get_prefered_co_actor(actor_name):
    
    max_w_ij = -float("inf")
    
    actor_movie_list = actor_movie_map[actor_name]
    
    for key, val in actor_movie_map.items():
        if key != actor_name:
            w_ij = len(set(actor_movie_list).intersection(set(val))) / len(actor_movie_list)
            
            if w_ij > max_w_ij:
                max_w_ij = w_ij
                preferred_actor = [(key, w_ij)]
                
            elif w_ij == max_w_ij:
                preferred_actor.append((key, w_ij))

    return preferred_actor 

<h2> Preferred Co-Actor </h2>

In [15]:
actor_pairings = ["Cruise, Tom", "Watson, Emma (II)", "Clooney, George", 
                  "Hanks, Tom", "Johnson, Dwayne (I)", "Depp, Johnny", 
                  "Smith, Will (I)", "Streep, Meryl", "DiCaprio, Leonardo", "Pitt, Brad"]

for i in actor_pairings:
    j = get_prefered_co_actor(i)
    print(i, " : ", j)

Cruise, Tom  :  [('Kidman, Nicole', 0.1746031746031746)]
Watson, Emma (II)  :  [('Radcliffe, Daniel', 0.52)]
Clooney, George  :  [('Damon, Matt', 0.11940298507462686)]
Hanks, Tom  :  [('Allen, Tim (I)', 0.10126582278481013)]
Johnson, Dwayne (I)  :  [('Austin, Steve (IV)', 0.20512820512820512), ('Calaway, Mark', 0.20512820512820512), ('Levesque, Paul (I)', 0.20512820512820512)]
Depp, Johnny  :  [('Bonham Carter, Helena', 0.08163265306122448)]
Smith, Will (I)  :  [('Foster, Darrell', 0.12244897959183673)]
Streep, Meryl  :  [('De Niro, Robert', 0.061855670103092786), ('Kline, Kevin (I)', 0.061855670103092786)]
DiCaprio, Leonardo  :  [('Scorsese, Martin', 0.10204081632653061)]
Pitt, Brad  :  [('Clooney, George', 0.09859154929577464)]


<h2> PageRank on Actor Network </h2>

In [17]:
top10_pagerank = open('top10_pagerank_actors.txt').read().split('\n')

print("Top 10 Page Rank scores")
print("\nActor Name - Number of Movies\n")
for i in top10_pagerank:
    if i:
        print(i, " - ", num_movies[i])

Top 10 Page Rank scores

Actor Name - Number of Movies

Flowers, Bess  -  828
Tatasciore, Fred  -  353
Harris, Sam (II)  -  600
Blum, Steve (IX)  -  373
Miller, Harold (I)  -  561
Jeremy, Ron  -  637
Phelps, Lee (I)  -  647
Lowenthal, Yuri  -  318
Downes, Robin Atkin  -  267
O'Connor, Frank (I)  -  623


<h2> A list of famous actors and their information </h2>

In [19]:
print("Famous Actors")

print("\nActor Name - Number of Movies\n")

for i in actor_pairings:
    if i:
        print(i, " - ", num_movies[i])

Famous Actors

Actor Name - Number of Movies

Cruise, Tom  -  63
Watson, Emma (II)  -  25
Clooney, George  -  67
Hanks, Tom  -  79
Johnson, Dwayne (I)  -  78
Depp, Johnny  -  98
Smith, Will (I)  -  49
Streep, Meryl  -  97
DiCaprio, Leonardo  -  49
Pitt, Brad  -  71


<h2> Creating movie network weights based on Jaccard score </h2>

In [15]:
movie_network_weights = open('movienetwork_weights.txt','a')

for pair, count in movie_pairings.items():
    
    num_actors_movie_i = len(movie_actor_map[pair[0]])
    num_actors_movie_j = len(movie_actor_map[pair[1]])
    
    if num_actors_movie_i >= 5 and num_actors_movie_j >= 5:

        intersection = count
        union = len(movie_actor_map[pair[0]]) + len(movie_actor_map[pair[1]]) - intersection
        w_ij = intersection/union
        
        if w_ij:
            movie_network_weights.write('\t'.join([pair[0], pair[1], str(w_ij)]))    
            movie_network_weights.write('\n') 

<h2> Save data as pickle object for future use </h2>

In [ ]:
# Saving the objects:
with open('actor_movie_map.pkl', 'wb') as f:
    pickle.dump([actor_movie_map], f)

with open('movie_actor_map.pkl', 'wb') as f:
    pickle.dump([movie_actor_map], f)
